# Projeto ADEL (Análise Dados Estadia em Limeira) - Webscraping das acomodações
Neste notebook, coletarei dados de acomodações anunciadas de algumas fontes. 

In [1]:
import pandas as pd

In [2]:
ss_sites = pd.read_csv('../Data/Imobiliarias/imobiliarias.csv', squeeze=True)

## Teste estático com BeautifulSoup

In [ ]:
from urllib.request import urlopen
import ssl

# This restores the same behavior as before.
context = ssl._create_unverified_context()
response = urlopen("url", context=context)
html_source = response.read()
soup = BeautifulSoup(html_source, 'html.parser')

## Coletando acomodações com Selenium

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
from lxml import etree
from urllib.request import urlopen
from datetime import date
import ssl
import hashlib
import webbrowser
import re
import time

In [4]:
def add_row_df(df, columns, row):
    row_sr = pd.Series(row, index=columns)
    row_df = pd.DataFrame([row_sr])
    df = pd.concat([row_df, df], ignore_index=True)
    return df

def hash_data(data):
    return hashlib.md5(data.encode()).hexdigest()

def goto_imovel_url(imovel_url):
    context = ssl._create_unverified_context()
    response = urlopen(imovel_url, context=context)
    html_source = response.read()
    return BeautifulSoup(html_source, 'html.parser')

def use_xpath(soup):
    return etree.HTML(str(soup))

def xpath_get_text(dom, xpath):
    texto = dom.xpath(xpath)[0].text
    return texto

In [ ]:
soup_imovel_url = goto_imovel_url('url')
dom = use_xpath(soup_imovel_url)

### Imobiliária 1/3

In [5]:
# Dataframes
df_columns = [
    'imob',
    'imovel_url',
    'codigo', 
    'nome', 
    'bairro', 
    'aluguel',
    'condominio',
    'area',
    'quartos',
    'banheiros',
    'vaga_garagem',
    'descricao'
]
df_acomodacoes = pd.DataFrame(data=None, columns=df_columns)
df_acomodacoes_hashcode = pd.DataFrame(data=None, columns=df_columns)

site_index = 0

# Hide chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')

#driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
driver.get(ss_sites[site_index])

# Acessar página de kitnets
time.sleep(3)
driver.find_element_by_xpath('/html/body/main/div[5]/section/div[1]/div/div/div/div/div[1]/div[3]/div/div').click()
driver.find_element_by_xpath('/html/body/main/div[5]/section/div[1]/div/div/div/div/div[1]/div[3]/div/ul/li[11]/a').click()
driver.find_element_by_xpath('/html/body/main/div[5]/section/div[1]/div/div/div/div/div[1]/div[7]/a/div').click()

# Coletar os dados das kitnets
time.sleep(3)
pag = 1

# Avançar nas páginas
try:
    while True:
        print()
        print('-'*60)
        print('Página {}'.format(pag))
        print('-'*60)
        
        # Coletar dados estaticamente com BeautifulSoup
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        soup_acomodacoes = soup.find_all('div', {'class': 'list__card jetgrid__col--6 jetgrid__col--md-8 jetgrid__col--sm-12 jetgrid__col--xs-24'})
        
        # Coletar dados de cada anúncio
        for num_acomodacao, acomodacao in enumerate(soup_acomodacoes):
            imovel_url = ss_sites[site_index] + acomodacao.find('a', {'class': 'list__link'})['href']
            cod = acomodacao.find('p', {'class': 'list__reference'}).get_text()
            nome = acomodacao.find('p', {'class': 'list__type'}).get_text()
            bairro = acomodacao.find('p', {'class': 'list__address'}).get_text()
            aluguel = acomodacao.find('div', {'class': 'list__price'}).get_text().strip()
            print(f'Acomodação: {num_acomodacao+1} de {len(soup_acomodacoes)}')
            print(bairro)
            print(aluguel)
            
            # Pegar os demais dados
            soup_imovel_url = goto_imovel_url(imovel_url)
            time.sleep(3.5)
            #dom = use_xpath(soup_imovel_url)
            
            # Adicionar manualmente
            webbrowser.open_new_tab(imovel_url)
            
            #condominio = float(input('Digite o valor do condomínio: '))
            condominio = 0
            area = int(input('Digite a área: '))
            num_quartos = 1
            num_banheiros = 1
            vaga_garagem = input('Digite se há vaga de garagem: ')
            descricao = soup_imovel_url.find('p', {'class': 'card__text'}).get_text()
            
            # Adicionar registro
            df_acomodacoes = add_row_df(df_acomodacoes, df_columns,
                                                   [ss_sites[site_index], 
                                                    imovel_url,
                                                    cod,
                                                    nome, 
                                                    bairro,
                                                    aluguel,
                                                    condominio,
                                                    area,
                                                    num_quartos,
                                                    num_banheiros,
                                                    vaga_garagem,
                                                    descricao])
            df_acomodacoes_hashcode = add_row_df(df_acomodacoes_hashcode, df_columns,
                                                            [hash_data(ss_sites[site_index]),
                                                             hash_data(imovel_url),
                                                             hash_data(cod), 
                                                             hash_data(nome), 
                                                             bairro,
                                                             aluguel,
                                                             condominio,
                                                             area,
                                                             num_quartos,
                                                             num_banheiros,
                                                             vaga_garagem,
                                                             descricao])
            print('*'*50)
        
        # Parar quando não encontrar próxima página
        if driver.find_element_by_xpath('/html/body/main/div[5]/section/div[3]/div[2]/div[9]/div/div/a[2]').get_attribute('href') == None:
            break
            
        # Avançar para a próxima página
        driver.find_element_by_xpath('/html/body/main/div[5]/section/div[3]/div[2]/div[9]/div/div/a[2]').click()
        pag += 1
        time.sleep(3)
        
except:
    print('Fim')


------------------------------------------------------------
Página 1
------------------------------------------------------------
Acomodação: 1 de 8
Jardim Cidade Universitaria I - Limeira/SP
Locação:  R$ 1.200,00
Digite a área: 25
Digite se há vaga de garagem: Sim
**************************************************
Acomodação: 2 de 8
Jardim Cidade Universitaria I - Limeira/SP
Locação:  R$ 1.200,00
Digite a área: 25
Digite se há vaga de garagem: Sim
**************************************************
Acomodação: 3 de 8
Jardim Cidade Universitaria I - Limeira/SP
Locação:  R$ 1.200,00
Digite a área: 20
Digite se há vaga de garagem: Sim
**************************************************
Acomodação: 4 de 8
Jardim Cidade Universitaria I - Limeira/SP
Locação:  R$ 1.200,00
Digite a área: 20
Digite se há vaga de garagem: Sim
**************************************************
Acomodação: 5 de 8
Jardim Cidade Universitaria I - Limeira/SP
Locação:  R$ 1.200,00
Digite a área: 20
Digite se há vaga 

### Imobiliária 2/3

In [ ]:
soup_test = goto_imovel_url('url')
dom_test = use_xpath(soup)

In [6]:
def scroll_command():
    js_scroll_command = "function getElementByXpath(path) {return document.evaluate(path, document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;}"
    js_scroll_command += "\nconst theElement = getElementByXpath('/html/body/main/section[1]/div[2]/div')"
    #js_scroll_command += "\nconst scrollToBottom = (node, scroll_step) => {node.scrollTo(0, 2000 * scroll_step);}"
    js_scroll_command += "\nconst scrollToBottom = (node, scroll_step) => {node.scrollTop = node.scrollHeight;}"
    js_scroll_command += f"\nscrollToBottom(theElement);"
    return js_scroll_command

In [7]:
site_index = 1

# Hide chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')

#driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
driver.get(ss_sites[site_index])

######################################
# Acessar página de kitnets
######################################
time.sleep(3)
# Select tipo
driver.find_element_by_xpath('/html/body/main/section[1]/div/div/div/div/div/div/form/div[1]/div[5]/div/div/button').click()
# kitnet
driver.find_element_by_xpath('/html/body/main/section[1]/div/div/div/div/div/div/form/div[1]/div[5]/div/div/div/ul/li[7]/a').click()
# Botão search
driver.find_element_by_id('submit-main-search-form').click()

# Coletar os dados das kitnets
time.sleep(3)
pag = 1
num_imoveis = 0
anuncios_encontrados = 0
anuncios = ''

# Coletar dados da página estaticamente com BeautifulSoup
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')
time.sleep(2)

# Coletar o número de imóveis encontrados
num_imoveis = soup.find('section', {'class': 'page-busca-resultado'}).find('h3').get_text()
num_imoveis = re.findall('[0-9]+', num_imoveis)
num_imoveis = int(num_imoveis[0])
print(f'num_imoveis: {num_imoveis}')

# Dar scroll até encontrar todos os imóveis
while anuncios_encontrados < num_imoveis:
    print()
    print('-'*50)
    print('Página {}'.format(pag))
    print('-'*50)
    pag += 1
    
    # Dar scroll
    driver.execute_script(scroll_command())
    time.sleep(2)
    
    # Recarregar página do soup
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(3)
    
    # Coletar os anuncios exibidos
    anuncios = soup.find('div', {'id': 'imovel-boxes'}).find_all('div', {'class': 'col-xs-12 imovel-box-single'})
    anuncios_encontrados = len(anuncios)
    
    print(f'anuncios_encontrados : {anuncios_encontrados} de {num_imoveis}')
    #print(anuncios[0].find('div', {'class': 'titulo-anuncio'}).find_all('p')[-1].get_text())
    #print(anuncios[-1].find('div', {'class': 'titulo-anuncio'}).find_all('p')[-1].get_text())

num_imoveis: 36

--------------------------------------------------
Página 1
--------------------------------------------------
anuncios_encontrados : 24 de 36

--------------------------------------------------
Página 2
--------------------------------------------------
anuncios_encontrados : 36 de 36


In [ ]:
# Dataframes
df_columns = [
    'imob',
    'imovel_url',
    'codigo', 
    'nome', 
    'bairro', 
    'aluguel',
    'condominio',
    'area',
    'quartos',
    'banheiros',
    'vaga_garagem',
    'descricao'
]
df_acomodacoes_hashcode_TEST = pd.DataFrame(data=None, columns=df_columns)

In [8]:
# Coletar dados de cada anúncio
for num_acomodacao, acomodacao in enumerate(anuncios):
    imovel_url = acomodacao.find('h3', {'class': 'titulo-grid'}).find('a')['href']
    cod = acomodacao.find('div', {'class': 'titulo-anuncio'}).find_all('p')[-1].get_text()
    nome = acomodacao.find('h3', {'class': 'titulo-grid'}).find('a').get_text()
    bairro = acomodacao.find('div', {'class': 'titulo-anuncio'}).find_all('a')[-1].get_text()
    aluguel = acomodacao.find('span', {'class': 'thumb-price'}).get_text()
    print(f'Acomodação: {num_acomodacao+1} de {num_imoveis}')
    print(bairro)
    print(aluguel)
    
    # Pegar os demais dados
    soup_imovel = goto_imovel_url(imovel_url)
    time.sleep(1)
    
    soup_amenities = soup_imovel.find('div', {'class': 'property-amenities'})
    condominio = 0 # Sem dados
    try:
        area = soup_amenities.find('div', {'id': 'amenity-area-privativa'}).find('span').get_text()
    except:
        webbrowser.open_new_tab(imovel_url)
        area = int(input('Digite a área: '))
    try:
        num_quartos = soup_amenities.find('div', {'id': 'amenity-dormitorios'}).find('span').get_text()
    except:
        #webbrowser.open_new_tab(imovel_url)
        #num_quartos = int(input('Digite o número de quartos: '))
        num_quartos = 1
    try:
        num_banheiros = soup_amenities.find('div', {'id': 'amenity-banheiros'}).find('span').get_text()
    except:
        #webbrowser.open_new_tab(imovel_url)
        #num_banheiros = int(input('Digite o número de banheiros: '))
        num_banheiros = 1
    vaga_garagem = 'null' # Sem dados
    descricao = soup_imovel.find('section', {'id': 'clb-descricao'}).find('div', {'class': 'col-xs-12 col-sm-12 col-md-7 col-lg-8'}).find('p').get_text()
    
    # Adicionar registro
    df_acomodacoes = add_row_df(df_acomodacoes, df_columns,
                                           [ss_sites[site_index], 
                                            imovel_url,
                                            cod,
                                            nome, 
                                            bairro,
                                            aluguel,
                                            condominio,
                                            area,
                                            num_quartos,
                                            num_banheiros,
                                            vaga_garagem,
                                            descricao])
    df_acomodacoes_hashcode = add_row_df(df_acomodacoes_hashcode, df_columns,
                                                    [hash_data(ss_sites[site_index]),
                                                     hash_data(imovel_url),
                                                     hash_data(cod), 
                                                     hash_data(nome), 
                                                     bairro,
                                                     aluguel,
                                                     condominio,
                                                     area,
                                                     num_quartos,
                                                     num_banheiros,
                                                     vaga_garagem,
                                                     descricao])
    print('*'*60)

Acomodação: 1 de 36
Jardim Cidade Universitária I - Limeira/Sp 
Sob consulta
************************************************************
Acomodação: 2 de 36
Jardim Cidade Universitária I - Limeira/Sp 
Sob consulta
************************************************************
Acomodação: 3 de 36
Jardim Cidade Universitária I - Limeira/Sp 
Sob consulta
************************************************************
Acomodação: 4 de 36
Jardim Morro Azul - Limeira/Sp 
R$ 700,00
************************************************************
Acomodação: 5 de 36
Jardim Morro Azul - Limeira/Sp 
R$ 700,00
************************************************************
Acomodação: 6 de 36
Jardim Cidade Universitária I - Limeira/Sp 
R$ 780,00
************************************************************
Acomodação: 7 de 36
Jardim Cidade Universitária I - Limeira/Sp 
R$ 800,00
************************************************************
Acomodação: 8 de 36
Jardim Cidade Universitária I - Limeira/Sp 
R$ 800

### Imobiliária 3/3

In [ ]:
soup_test = goto_imovel_url('url')
dom_test = use_xpath(soup)

In [ ]:
# Dataframes
df_columns = [
    'imob',
    'imovel_url',
    'codigo', 
    'nome', 
    'bairro', 
    'aluguel',
    'condominio',
    'area',
    'quartos',
    'banheiros',
    'vaga_garagem',
    'descricao'
]
df_acomodacoes_hashcode_TEST = pd.DataFrame(data=None, columns=df_columns)

In [9]:
site_index = 2

# Hide chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')

#driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
driver.get(ss_sites[site_index])

# Acessar página de kitnets
time.sleep(3)
driver.find_element_by_xpath('//*[@id="busca1"]/form/div/div[1]/select').click()
driver.find_element_by_xpath('//*[@id="busca1"]/form/div/div[1]/select/option[9]').click()
driver.find_element_by_xpath('//*[@id="busca1"]/form/input[3]').click()

# Coletar os dados das kitnets
time.sleep(3)
pag = 1

# Avançar nas páginas
try:
    while True:
        print()
        print('-'*50)
        print('Página {}'.format(pag))
        print('-'*50)
        
        # Coletar dados estaticamente com BeautifulSoup
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        soup_acomodacoes = soup.find_all('div', {'class': 'imovel'})
        
        # Coletar dados de cada anúncio
        for num_acomodacao, acomodacao in enumerate(soup_acomodacoes):
            imovel_url = ss_sites[site_index] + acomodacao.find('div', {'class': 'imovelBotao'}).find('a')['href']
            cod = acomodacao.find('div', {'class': 'imovelChave'}).get_text().split('|')[0]
            nome = acomodacao.find('div', {'class': 'imovelInfo'}).find('h2').get_text().split('|')[0]
            bairro = acomodacao.find('div', {'class': 'imovelInfo'}).find('h2').get_text().split('|')[-1]
            aluguel = acomodacao.find('div', {'class': 'imovelChave'}).get_text().split('|')[-1].split()[-1]
            
            num_quartos = acomodacao.find('div', {'class': 'imovelIcones'}).find('div').get_text()
            print(f'Acomodação: {num_acomodacao+1} de {len(soup_acomodacoes)}')
            print(bairro)
            print(aluguel)
            
            # Pegar os demais dados
            soup_imovel = goto_imovel_url(imovel_url)
            time.sleep(1)
            
            condominio = soup_imovel.find('div', {'id': 'imovelExtra'}).get_text()
            area = soup_imovel.find('div', {'class': 'bloco'}).find_all('li')[0].get_text()
            # Numero de quartos e vagas de garagem
            num_banheiros = ''
            vaga_garagem = ''
            for i in range(len(soup_imovel.find('div', {'id': 'imovelIcones'}).find_all('div'))):
                if soup_imovel.find('div', {'id': 'imovelIcones'}).find_all('div')[i].find('span').get_text() == 'Banheiro':
                    num_banheiros = soup_imovel.find('div', {'id': 'imovelIcones'}).find_all('div')[i].find('strong').get_text()
                if soup_imovel.find('div', {'id': 'imovelIcones'}).find_all('div')[i].find('span').get_text() == 'Vaga':
                    vaga_garagem = soup_imovel.find('div', {'id': 'imovelIcones'}).find_all('div')[i].find('strong').get_text()
            if num_banheiros == '':
                #webbrowser.open_new_tab(imovel_url)
                #num_banheiros = int(input('Digite o número de banheiros: '))
                num_banheiros = 1
            if vaga_garagem == '':
                vaga_garagem = 0 # Gabarito

            descricao = soup_imovel.find('div', {'id': 'imovelDescricao'}).find('p').get_text()
            for desc in soup_imovel.find('div', {'id': 'imovelDescricao'}).find_all('li'):
                descricao += ("\n" + desc.get_text())
            
            # Adicionar registro
            df_acomodacoes = add_row_df(df_acomodacoes, df_columns,
                                                   [ss_sites[site_index], 
                                                    imovel_url,
                                                    cod,
                                                    nome, 
                                                    bairro,
                                                    aluguel,
                                                    condominio,
                                                    area,
                                                    num_quartos,
                                                    num_banheiros,
                                                    vaga_garagem,
                                                    descricao])
            df_acomodacoes_hashcode = add_row_df(df_acomodacoes_hashcode, df_columns,
                                                            [hash_data(ss_sites[site_index]),
                                                             hash_data(imovel_url),
                                                             hash_data(cod), 
                                                             hash_data(nome), 
                                                             bairro,
                                                             aluguel,
                                                             condominio,
                                                             area,
                                                             num_quartos,
                                                             num_banheiros,
                                                             vaga_garagem,
                                                             descricao])
            
            print('*'*50)
        
        # Parar quando não encontrar próxima página
        if driver.find_element_by_xpath('//*[@id="pagenav"]/nav/ul/li[5]/a').get_attribute('href') == None:
            break
            
        # Avançar para a próxima página
        driver.find_element_by_xpath('//*[@id="pagenav"]/nav/ul/li[6]/a').click() # Ficar de olho no menu
        pag += 1
        time.sleep(2)
        
except:
    print('Fim')


--------------------------------------------------
Página 1
--------------------------------------------------
Acomodação: 1 de 10
 CIDADE UNIVERSITARIA
R$700,00
**************************************************
Acomodação: 2 de 10
 CIDADE UNIVERSITARIA
R$700,00
**************************************************
Acomodação: 3 de 10
 CIDADE UNIVERSITARIA
R$800,00
**************************************************
Acomodação: 4 de 10
 JD. PAULISTA
R$950,00
**************************************************
Acomodação: 5 de 10
 CIDADE UNIVERSITARIA
R$950,00
**************************************************
Acomodação: 6 de 10
 CIDADE UNIVERSITARIA
R$950,00
**************************************************
Acomodação: 7 de 10
 JD. PAULISTA
R$950,00
**************************************************
Acomodação: 8 de 10
 JD. PAULISTA
R$950,00
**************************************************
Acomodação: 9 de 10
 JD. PAULISTA
R$950,00
**************************************************
Ac

In [10]:
df_acomodacoes_hashcode.shape

(84, 12)

In [11]:
df_acomodacoes_hashcode.sample(5)

,imob,imovel_url,codigo,nome,bairro,aluguel,condominio,area,quartos,banheiros,vaga_garagem,descricao
20,c7234506476bbf0aff48eda764ff9eba,8281ee9fcc5eba6b80e3975d7692515a,b3b477f0632310b03bde750706df4596,245d9e8bab947ecf1ee8867fe9597ab2,JD. PAULISTA,"R$950,00","\n\n + Condomínio R$0,00 i\...",16 M2,1,1,0,Kitnet localizada próximo à Unicamp (FCA) cont...
25,c7234506476bbf0aff48eda764ff9eba,de9565da54c168ae203ff173560c80b9,48cf0deecd859f082300f8bb8f2ee7c6,245d9e8bab947ecf1ee8867fe9597ab2,CIDADE UNIVERSITARIA,"R$950,00","\n\n + Condomínio R$100,00 ...",25 M2,1,1,1,Apartamento localizado no Chacara Antonieta co...
19,c7234506476bbf0aff48eda764ff9eba,25339b5751bc16c25b6bdca819691a40,361a537a61eaa8e33466a2a8bbe7c274,245d9e8bab947ecf1ee8867fe9597ab2,CIDADE UNIVERSITARIA,"R$1.000,00","\n\n + Condomínio R$0,00 i\...",,1,1,0,Kitnet próxima à Unicamp (FCA) contendo quarto...
64,ef23a7e1738f4b316011bbdd88e514a2,172fdf6fc7dcf08ee3c1faed4842685e,9b3115053caa712164160d579c06be48,0ef98b2b924d268967e63d0747a4bd65,Jardim Cidade Universitária I - Limeira/Sp,Sob consulta,0,0 m2,1,1,null,"Kitnet com cozinha, um quarto, banheiro e área..."
13,c7234506476bbf0aff48eda764ff9eba,e034ef50a28d4f738a0bd4068f761724,a264ee68837c839ccf3a90ee71c91b41,245d9e8bab947ecf1ee8867fe9597ab2,CIDADE UNIVERSITARIA,"R$1.000,00","\n\n + Condomínio R$0,00 i\...",35 M2,1,1,0,Kitnet próxima à Unicamp (FCA) contendo quarto...


In [14]:
today = date.today()
current_date = today.strftime("%d_%m_%Y")
filename = 'acomodacoes_' + current_date + '.csv'
filename_hashcode = 'webscraping_acomodacoeshash_' + current_date + '.csv'

df_acomodacoes.to_csv('../Data/Imobiliarias/' + filename, index=False)
df_acomodacoes_hashcode.to_csv('../datalake/landing/webscraping-acomodacoes/' + filename_hashcode, index=False)